In [1]:
#Libraries
import mysql.connector
import pandas as pd
from datetime import datetime, timedelta, date, time
from tqdm import tqdm
from sqlalchemy import create_engine
from sqlalchemy import text
import pymysql
pymysql.install_as_MySQLdb()
import requests


#For ZIpCodes
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

## DB Coonnect

In [2]:
database = "sd_pd_calls"
user = 'root'
password = 'Password1'
host = 'localhost'
GOOGLE_API_KEY = 'XXXXXX'
user_agent_name = 'Sal-App'

In [3]:
#Initial Connection
cnx = mysql.connector.connect(user=user,
                              password=password,
                              host=host,
                              database=database
                              )
cursor = cnx.cursor()

##  Find if Row exists

In [4]:
def Find_Data(row,cursor,table, key):
    try:
        #print(f"SELECT * from {table} where {key} = '{row[key]}'")
        cursor.execute(f"SELECT * from {table} where {key} = '{row[key]}'")
        # Fetch a record
        result = cursor.fetchone()
    except:
        result = 0
    
    return result

## Load Variables

In [5]:
#Calls
url = f"http://seshat.datasd.org/pd/pd_cfs_calltypes_datasd.csv"
call_type = pd.read_csv(url).dropna(axis=1)


In [6]:
#Dispos
url = f"http://seshat.datasd.org/pd/pd_dispo_codes_datasd.csv"
disposition = pd.read_csv(url).dropna(axis=1)
disposition.loc[len(disposition.index)] = ['DEF', 'DEF'] 
disposition.loc[len(disposition.index)] = ['SS-', 'SS-'] 

In [7]:
#Beats
beat = pd.read_csv('beat-neighborhood.csv').dropna(axis=1)
beat_split = beat['beat,neighborhood'].str.split(pat=",",
                                                 expand=True)
beat_split = beat_split.rename(columns={0: "beat",
                                        1: "neighborhood"})
beat_split.loc[len(beat_split.index)] = [-1, 'No Beat']
beat_split.loc[len(beat_split.index)] = [700, '700'] 
beat_split.loc[len(beat_split.index)] = [63, '63']
beat_split.loc[len(beat_split.index)] = [64, '64'] 
beat_split.loc[len(beat_split.index)] = [0, '0'] 
beat_split.loc[len(beat_split.index)] = [300, '300']
beat_split.loc[len(beat_split.index)] = [9, '9'] 


In [8]:
# Create dataframe
days = pd.DataFrame({
    'day_of_week':list(range(1,8)),
    'day':['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'],
})


In [9]:
# df to dict
call_type_dict = call_type.to_dict('records')
disposition_dict = disposition.to_dict('records')
beat_dict = beat_split.to_dict('records')
days_dict = days.to_dict('records')

In [10]:
# SQL Variable Tables
SQL_Tables = pd.DataFrame({
    'Table':['Call_types','Dispositions','Beats'],
    'Key':['call_type','dispo_code','beat'],
    'Col2':['description','description','neighborhood'],
    'dict':[call_type_dict,disposition_dict,beat_dict]
})
SQL_Tables_dict = SQL_Tables.to_dict('records')

In [11]:
#Load Data
def Load_Data(row,cursor,table,Col1,Col2):
    # Create a new record
    sql = f"""INSERT INTO {table} ({Col1},
                                    {Col2}
                                    ) VALUES (%s, %s)"""

    # Execute the query
    cursor.execute(sql, (row[Col1],row[Col2]))
    cnx.commit()  

In [12]:
for upper_row in tqdm(SQL_Tables_dict):
    for row in upper_row['dict']:
        Data_Found = Find_Data(row,cursor,
                               upper_row['Table'],
                               upper_row['Key'])
        if Data_Found is None:
            #If no data found load 
            Load_Data(row,
                      cursor,
                      upper_row['Table'],
                      upper_row['Key'],
                      upper_row['Col2'])
        else:
            pass

100%|██████████| 3/3 [00:00<00:00,  8.35it/s]


# Load Historical Calls

Loading Last 4 days

In [13]:
#Need to create or import current year by default
year = datetime.now().year;

url = f"https://seshat.datasd.org/pd/pd_calls_for_service_{year}_datasd.csv"
df = pd.read_csv(url)

#changing format from object to datetime
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d')

#Using current date minus 4 days to get recent data
current_date = date.today()
df_recent = df.loc[(df['date_time'].dt.date == current_date - timedelta(4))]

In [14]:
#Verify minimum and maximum dates
df_recent.max()
#df_recent.min()

C:\Users\ebbi_\AppData\Local\Temp\ipykernel_45704\2498810194.py:2: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_recent.max()


incident_num                       E23020006265
date_time                   2023-02-04 23:59:15
day_of_week                                   7
address_number_primary                    34800
address_road_primary                        ZOO
address_dir_intersecting                    NaN
address_sfx_intersecting                    NaN
call_type                               WARRANT
disposition                                   X
beat                                        937
priority                                      9
dtype: object

### Load Calls

In [15]:
#Calls
def Load_Calls(row,cursor,table,year):
    # Create a new record
    sql = f"""INSERT INTO {table} ( incident_num,
                                    incident_date_time,
                                    call_type,
                                    dispo_code,
                                    beat,
                                    priority,
                                    incident_year
                                    ) VALUES (%s,%s,%s,%s,%s,%s,%s)"""

    # Execute the query
    cursor.execute(sql, (row['incident_num'],
                         row['date_time'],
                         row['call_type'],
                         row['disposition'],
                         row['beat'],
                         row['priority'],
                         year,
                        )
                  )
    cnx.commit()  

In [16]:
recent_data = df_recent
recent_data_dict = recent_data.to_dict('records')

for row in tqdm(recent_data_dict):
    #Load Service Calls
    Table = 'Service_calls'
    Key = 'incident_num'
    #Verify if Data is in table
    Data_Found = Find_Data(row,cursor,Table,Key)
    if Data_Found is None:
        #If no data found load
        try:
            Load_Calls(row,cursor,Table,year)
        except:
            print(row)
            
    else:
        pass

100%|██████████| 1282/1282 [00:00<00:00, 1428.21it/s]


### Load Address

In [17]:
#Address
def Load_Calls_Address(row,cursor,table):
    address_list = [row['address_number_primary'],
               row['address_dir_primary'],
               row['address_road_primary'],
               row['address_sfx_primary'],
                    'San Diego CA'
                   ]
    
    def concatenate_elements(elements):
        result = ''
        for element in elements:
            if element is not None and not pd.isnull(element) and element != 0:
                result += str(element) + ' '
        return result.strip()
    
    
    address = concatenate_elements(address_list)
    
    # Create a new record
    sql = f"""INSERT INTO {table} ( incident_num,
                                    address_number_primary,
                                    address_dir_primary,
                                    address_road_primary,
                                    address_sfx_primary,
                                    address_dir_intersecting,
                                    address_road_intersecting,
                                    address_sfx_intersecting,
                                    address
                                    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

    # Execute the query
    cursor.execute(sql, (row['incident_num'],
                         row['address_number_primary'],
                         row['address_dir_primary'],
                         row['address_road_primary'],
                         row['address_sfx_primary'],
                         row['address_dir_intersecting'],
                         row['address_road_intersecting'],
                         row['address_sfx_intersecting'],
                         address
                        )
                  )
    cnx.commit()

In [18]:
#Load Address

recent_data = df_recent
recent_data_dict = recent_data.to_dict('records')
for row in tqdm(recent_data_dict):
    #Load Service Calls Address        
    Table = 'Address'
    Key = 'incident_num'
    #Verify if Data is in table
    Data_Found = Find_Data(row,cursor,Table,Key)
    if Data_Found is None:
        #If no data found load
        try:
            pass
            #Load_Calls_Address(row,cursor,Table)
        except:
            pass
            #print(row)
            
    else:
        pass

100%|██████████| 1282/1282 [00:00<00:00, 1592.68it/s]


## Need to improve the address loading, droping to many

### Lat and Long

In [19]:
#Connect to DB
engine = create_engine(f"mysql://{user}:{password}@{host}/{database}")
connection = engine.connect()

In [20]:
result = connection.execute(text("SELECT incident_num, address from Address"))
column_names = result.keys()
rows = result.fetchall()
result_df = pd.DataFrame(rows, columns=column_names)
Address_Book = result_df.head(100)
Address_Book

,incident_num,address


In [21]:
def Lat_Long(address):
    lat, lng  = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address}&key={api_key}"
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        #error
        return None, None
    try:
        #found
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
        
    except:
        pass
    return lat, lng

def DF_GeoCode(row):
    column_name = 'address'
    address_value = row[column_name]
    address_lat, address_lng = Lat_Long(address_value)
    row['lat'] = address_lat
    row['lng'] = address_lng
    
    return row

In [22]:
Address_Book = Address_Book.apply(DF_GeoCode, axis=1)
Address_Book['lat'] = '0000'
Address_Book['lng'] = '0000'

In [23]:
#Lat and Long
def Load_LL(row,cursor,table):
    # Create a new record
    sql = f"""INSERT INTO {table} ( incident_num,
                                    lat,
                                    lng
                                    ) VALUES (%s,%s,%s)"""

    # Execute the query
    cursor.execute(sql, (row['incident_num'],
                         row['lat'],
                         row['lng']
                        )
                  )
    cnx.commit() 

In [24]:
for row in tqdm(Address_Book.to_dict('records')):
    Data_Found = Find_Data(row,
                           cursor,
                           'GeoLocations',
                           'incident_num')

    if Data_Found is None:
        #If no data found load 
        Load_LL(row,
                  cursor,
                  'GeoLocations')
    else:
        pass

0it [00:00, ?it/s]


### ZipCode

In [25]:
result = connection.execute(text("SELECT incident_num,lat,lng from GeoLocations"))
column_names = result.keys()
rows = result.fetchall()
ZipCode_Book = pd.DataFrame(rows, columns=column_names)
ZipCode_Book = ZipCode_Book.head(100)
ZipCode_Book

,incident_num,lat,lng


In [26]:
def get_zipcode(df, geolocator, lat_field, lon_field):
    try:
        location = geolocator.reverse((df[lat_field], df[lon_field]))
        result = location.raw['address']['postcode']
    except:
        result = None
    return result


In [27]:
geolocator = Nominatim(user_agent = user_agent_name)
zipcodes = ZipCode_Book.apply(get_zipcode,
                            axis=1,
                            geolocator=geolocator,
                            lat_field='lat',
                            lon_field='lng'
                           )

In [28]:
ZipCode_Book['Zipcode'] = zipcodes
ZipCode_Book

,incident_num,lat,lng,Zipcode


In [29]:
#Lat and Long
def Load_Zipcode(row,cursor,table):
    # Create a new record
    sql = f"""INSERT INTO {table} ( incident_num,
                                    Zipcode
                                    ) VALUES (%s,%s)"""

    # Execute the query
    cursor.execute(sql, (row['incident_num'],
                         row['Zipcode']
                        )
                  )
    cnx.commit() 

In [30]:
for row in tqdm(ZipCode_Book.to_dict('records')):
    Data_Found = Find_Data(row,
                           cursor,
                           'Zipcodes',
                           'incident_num')

    if Data_Found is None:
        #If no data found load 
        Load_Zipcode(row,
                  cursor,
                  'Zipcodes')
    else:
        pass

0it [00:00, ?it/s]
